# This notebook generates CMG dat files for running CMG simulations

# Step 1: set up a base CMG model
Prepare a base CMG dat file and add it to the wrtcmgdat.py

# Step 2: sample uncertain parameters

## 1. CMG requires initializating stress state using a reference block. For the JD_Sula_2005_gmc grid, there are 10 k layers. Reservoir starts at k=6. Use block (50, 1, 6) as reference block for *STRESSGRAD calculation. Its grid top = 670.7188 m and bottom = 671.9521 m.
## 2. PORO/PERMX pairs are NOT sampled more than once (from file names instead of files)

In [23]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from pathlib import Path
import sys
import re

# Setup for sampling
random_seed = 11
# Note: 1) stress gradients are effective ones (required by CMG) after subtracting 10; 
#       2) stress gradients are negative due to CMG DIR DOWN convention
params = ['E_GPa', 'PR', 'SH_MPa/km', 'Sh_MPa/km', 'Sv_MPa/km', 'SH_azi_deg']
# l_bounds = [15e6, 0.2, 25 * 0.9, 14.6 * 0.9, 22.7 * 0.9, 290]
# u_bounds = [25e6, 0.4, 25 * 1.1, 14.6 * 1.1, 22.7 * 1.1, 310]
l_bounds = [15e6, 0.2, -15 * 1.1, -4.6 * 1.1, -12.7 * 1.1, 290]
u_bounds = [25e6, 0.4, -15 * 0.9, -4.6 * 0.9, -12.7 * 0.9, 310]
num_samples = 90  # Change as needed

# Load PORO and PERMX file names
# property_file_names = np.load('property_file_names.npy')
property_file_names = np.loadtxt("property_file_names.csv",delimiter=",",dtype=str)

# sort the file names by the number in the name
def extract_number(filename):
    match = re.search(r"(\d+)", filename)
    return int(match.group(1)) if match else float('inf')

poro_file_names = sorted(
    [name for name in property_file_names if "PORO" in name.upper()],
    key=extract_number
)

permx_file_names = sorted(
    [name for name in property_file_names if "PERMX" in name.upper()],
    key=extract_number
)

# check a few things
if not poro_file_names or not permx_file_names:
    print("Error: PORO or PERMX file names not found.")
    sys.exit(1)

if len(poro_file_names) != len(permx_file_names):
    raise ValueError(f"Number of PORO file names ({len(poro_file_names)}) does not match number of PERMX file names ({len(permx_file_names)})")

num_pairs = len(poro_file_names)

if num_samples > num_pairs:
    raise ValueError(f"Cannot sample {num_samples} unique poro/permx pairs: only {num_pairs} available.")

# Latin Hypercube Sampling for parameters
sampler = qmc.LatinHypercube(d=len(params), seed=random_seed)
sample = sampler.random(n=num_samples)
sample_params = qmc.scale(sample, l_bounds, u_bounds)
df_params = pd.DataFrame(np.round(sample_params, 2), columns=params)

# Store poro/permx pairs
df_params["PORO_file"] = [str(poro_file_names[i]) for i in range(num_samples)]
df_params["PERMX_file"] = [str(permx_file_names[i]) for i in range(num_samples)]

# add prefix to file names
prefix = "data_properties/"
df_params["PORO_file"] = df_params["PORO_file"].apply(lambda x: f"{prefix}{x}")
df_params["PERMX_file"] = df_params["PERMX_file"].apply(lambda x: f"{prefix}{x}")

# Calculate stress state parameters
df_params['beta'] = df_params['SH_azi_deg'] - 90  # Rotate from SH to x-axis
df_params['cos_2beta'] = np.cos(np.radians(2 * df_params['beta']))
df_params['sin_2beta'] = np.sin(np.radians(2 * df_params['beta']))
# calculate the stress gradients in kPa/km
df_params['sigma_x_grad'] = (df_params['SH_MPa/km'] + df_params['Sh_MPa/km']) / 2 + \
                       (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['cos_2beta']
df_params['sigma_y_grad'] = (df_params['SH_MPa/km'] + df_params['Sh_MPa/km']) / 2 - \
                       (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['cos_2beta']
# tau_xy_grad should be positive after checking the directions of maximum stress in the CMG Results
df_params['tau_xy_grad'] = -(df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['sin_2beta']
# calculate the stress state for the reference block in kPa
# for the JD_Sula_2005_gmc grid, the reference block is (50, 1, 6) 
grid_top = 670.7188; grid_bottom = 671.9521; grid_ave = (grid_top + grid_bottom)/2
df_params['sigma_x_ref'] = df_params['sigma_x_grad'] * grid_ave *(-1)
df_params['sigma_y_ref'] = df_params['sigma_y_grad'] * grid_ave *(-1)
df_params['sigma_z_ref'] = df_params['Sv_MPa/km'] * grid_ave *(-1)
df_params['tau_xy_ref'] = df_params['tau_xy_grad'] * grid_ave *(-1)

# Output
df_params.to_csv("sampled_parameters_and_files.csv", index=False)
df_params


,E_GPa,PR,SH_MPa/km,Sh_MPa/km,Sv_MPa/km,SH_azi_deg,PORO_file,PERMX_file,beta,cos_2beta,sin_2beta,sigma_x_grad,sigma_y_grad,tau_xy_grad,sigma_x_ref,sigma_y_ref,sigma_z_ref,tau_xy_ref
0,17430158.87,0.32,-15.82,-4.71,-12.37,296.90,data_properties/JD_BASECASE_5_PORO.dat,data_properties/JD_BASECASE_5_PERMX.dat,206.90,0.590606,0.806960,-13.545814,-6.984186,4.482665,9093.785462,4688.731327,8304.419517,-3009.371612
1,18547731.05,0.23,-14.96,-4.30,-12.80,300.55,data_properties/JD_BASECASE_6_PORO.dat,data_properties/JD_BASECASE_6_PERMX.dat,210.55,0.483282,0.875465,-12.205895,-7.054105,4.666226,8194.250081,4735.670686,8593.093760,-3132.602884
2,21148573.01,0.27,-14.37,-4.63,-12.92,308.11,data_properties/JD_BASECASE_7_PORO.dat,data_properties/JD_BASECASE_7_PERMX.dat,218.11,0.238194,0.971217,-10.660007,-8.339993,4.729829,7156.440588,5598.932962,8673.654014,-3175.301981
3,15242584.84,0.31,-13.70,-4.34,-11.98,297.23,data_properties/JD_BASECASE_8_PORO.dat,data_properties/JD_BASECASE_8_PERMX.dat,207.23,0.581271,0.813710,-11.740349,-6.299651,3.808162,7881.712542,4229.178976,8042.598691,-2556.554413
4,18738525.02,0.32,-15.67,-4.52,-13.77,291.53,data_properties/JD_BASECASE_9_PORO.dat,data_properties/JD_BASECASE_9_PERMX.dat,201.53,0.730639,0.682764,-14.168313,-6.021687,3.806409,9511.690823,4042.571912,9244.289146,-2555.376968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,23335499.95,0.31,-15.57,-4.43,-12.77,301.18,data_properties/JD_BASECASE_194_PORO.dat,data_properties/JD_BASECASE_194_PERMX.dat,211.18,0.463915,0.885880,-12.584004,-7.415996,4.934351,8448.088238,4978.620762,8572.953697,-3312.604859
86,24592116.34,0.30,-15.92,-4.19,-12.21,291.78,data_properties/JD_BASECASE_197_PORO.dat,data_properties/JD_BASECASE_197_PERMX.dat,201.78,0.724653,0.689114,-14.305091,-5.804909,4.041652,9603.514441,3897.041459,8197.005845,-2713.304591
87,16276499.15,0.30,-16.40,-4.24,-13.53,302.25,data_properties/JD_BASECASE_198_PORO.dat,data_properties/JD_BASECASE_198_PERMX.dat,212.25,0.430511,0.902585,-12.937507,-7.702493,5.487719,8685.407398,5170.956290,9083.168638,-3684.099988
88,20519907.95,0.39,-15.63,-4.44,-12.94,297.56,data_properties/JD_BASECASE_199_PORO.dat,data_properties/JD_BASECASE_199_PERMX.dat,207.56,0.571860,0.820352,-13.234554,-6.835446,4.589867,8884.825393,4588.877088,8687.080723,-3081.340347


In [24]:
np.savetxt("property_file_names.csv",property_file_names,delimiter=",",fmt="%s")
property_file_names_txt = np.loadtxt("property_file_names.csv",delimiter=",",dtype=str)
property_file_names_txt.shape

(180,)

# Step3: generate CMG dat files based on the sampled parameters

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from pathlib import Path
import os
import sys
import time

# append the path of the parent directory
sys.path.append("..")
# Define the folder path to store generated dat files
folder_path = Path('../data')

In [4]:
# load the LHS sampled parameters and file names
df = pd.read_csv('sampled_parameters_and_files.csv')
df_input = df.iloc[:,[0,1,4,6,7,11,12,13]]
df_input

,E_GPa,PR,Sv_MPa/km,PORO_file,PERMX_file,sigma_x,sigma_y,tau_xy
0,22719207.98,0.37,21.71,data_properties/JD_BASECASE_21_PORO.dat,data_properties/JD_BASECASE_21_PERMX.dat,23.755719,16.414281,5.052311
1,20908977.31,0.26,24.80,data_properties/JD_BASECASE_22_PORO.dat,data_properties/JD_BASECASE_22_PERMX.dat,22.784531,14.985469,4.164215
2,18158073.63,0.34,23.67,data_properties/JD_BASECASE_26_PORO.dat,data_properties/JD_BASECASE_26_PERMX.dat,24.060823,18.099177,5.333554
3,16111690.09,0.21,22.68,data_properties/JD_BASECASE_27_PORO.dat,data_properties/JD_BASECASE_27_PERMX.dat,21.409079,18.440921,4.073050
4,23420097.04,0.30,21.04,data_properties/JD_BASECASE_28_PORO.dat,data_properties/JD_BASECASE_28_PERMX.dat,19.918173,17.731827,3.852921


In [ ]:
from utils.pyCMG_Model import omv_CCS

omvccs = omv_CCS()
omvccs.folder_path = folder_path
omvccs.title1 = 'pyCCUS testdrive'
omvccs.title2 = 'CCS omv'
omvccs.title3 = 'JD+YL'

omvccs.write_simfiles(df_input=df_input, verbose=True)

Job done -- write 5 CMG dat files based on exp design csv .....


# Step 4: run CMG simulations

## Option 1: run CMG on TS machines

In [7]:
# Record the start time
start_time = time.time()

from utils.pyCMG_Control import pycmgcontrol

for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10', 'ese-ts2win-v2024.20'
    pycmg_ctrl.cmg_version = 'ese-ts2win-v2024.20'
    pycmg_ctrl.run_gem_simulation(case_name_suffix=f'case{nn+1}.dat')
    
# Record the end time
end_time = time.time()

# Calculate the running time
elapsed_time = (end_time - start_time)/60
print(f"Elapsed time: {elapsed_time:.2f} minutes")

Elapsed time: 52.20 minutes


## Option 2: run CMG on Sherlock
- You will also need submit.sh file

In [ ]:
# sherlock = pysherlock()
# pyCTRL_folder_path = os.path.join(folder_path, 'pyCTRLfiles')

# for idx in range(df_input.shape[0]):
#     sherlock.write_pyCTRLfile(folder_path=pyCTRL_folder_path, caseid=idx+1)

# Step 5: convert simulation results into npy format

## Saturation and pressure

In [ ]:
n_cases = 2

from utils.pyCMG_Control import pycmgcontrol

for nn in range(n_cases):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10'
    pycmg_ctrl.cmg_version = 'ese-ts2win-v2024.20'
    pycmg_ctrl.rwd_precis = 4
    pycmg_ctrl.proplist = ['SG','PRES']
    pycmg_ctrl.layer_nums = [i for i in range(41,80)]
    pycmg_ctrl.time_query = [2030, 2040, 2050, 2060, 2550, 3050]
    ##### Params to control rwo2npy steps ######
    pycmg_ctrl.XY2arr_interp_method = "cubic"  # options = {‘linear’, ‘nearest’, ‘cubic’}
    pycmg_ctrl.XY2arr_interp_num_x = 107
    pycmg_ctrl.XY2arr_interp_num_y = 117
    pycmg_ctrl.x_dir_key = 'X'
    pycmg_ctrl.y_dir_key = 'Y'

    pycmg_ctrl.cmgrst2npy(caseid=f"{nn+1}", verbose=False, rwodelete=True)
#     npy_data = pycmg_ctrl.cmg2npy #a nested list consisted of all data, e.g., SG&PRES together 

## Geomechanical parameters

In [5]:
from utils.pyCMG_Control import pycmgcontrol

for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10'
    pycmg_ctrl.cmg_version = 'ese-ts2win-v2024.20'
    pycmg_ctrl.rwd_precis = 4
    pycmg_ctrl.proplist = ['Vertical Displacement from Geomechanics']
    pycmg_ctrl.layer_nums = [i for i in range(1,12)]
    pycmg_ctrl.time_query = [2024, 2026, 2028, 2030, 2032, 2034]
    ##### Params to control rwo2npy steps ######
    pycmg_ctrl.XY2arr_interp_method = "cubic"  # options = {‘linear’, ‘nearest’, ‘cubic’}
    pycmg_ctrl.XY2arr_interp_num_x = 100
    pycmg_ctrl.XY2arr_interp_num_y = 100
    pycmg_ctrl.x_dir_key = 'X'
    pycmg_ctrl.y_dir_key = 'Y'

    pycmg_ctrl.cmgrst2npy(caseid=f"{nn+1}", verbose=False, rwodelete=True)
#     npy_data = pycmg_ctrl.cmg2npy #a nested list consisted of all data, e.g., SG&PRES together 

In [6]:
from utils.pyCMG_Control import pycmgcontrol

for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10'
    pycmg_ctrl.cmg_version = 'ese-ts2win-v2024.20'
    pycmg_ctrl.rwd_precis = 4
    pycmg_ctrl.proplist = ['GEORTYPE']
    pycmg_ctrl.layer_nums = [i for i in range(1,12)]
    pycmg_ctrl.time_query = [2024, 2026, 2028, 2030, 2032, 2034]
    ##### Params to control rwo2npy steps ######
    pycmg_ctrl.XY2arr_interp_method = "cubic"  # options = {‘linear’, ‘nearest’, ‘cubic’}
    pycmg_ctrl.XY2arr_interp_num_x = 100
    pycmg_ctrl.XY2arr_interp_num_y = 100
    pycmg_ctrl.x_dir_key = 'X'
    pycmg_ctrl.y_dir_key = 'Y'

    pycmg_ctrl.cmgrst2npy(caseid=f"{nn+1}", verbose=False, rwodelete=True)
#     npy_data = pycmg_ctrl.cmg2npy #a nested list consisted of all data, e.g., SG&PRES together 

In [21]:
arr1=np.array(npy_data)
arr1.shape

(2, 100, 100, 3, 6)

In [10]:
l1=[i for i in range(1,12)]
l1

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

# Debug code

In [ ]:
   def read_VERDSPLGEO_rwo2npy(self, case_name, save=True):
        cmgrst = pycmgresults()
        cmgrst.XY2arr_interp_method = self.XY2arr_interp_method
        cmgrst.XY2arr_interp_num_x = self.XY2arr_interp_num_x
        cmgrst.XY2arr_interp_num_y = self.XY2arr_interp_num_y

        rwo_dir = os.path.join(self.simfolder, self.batchfolder, f'rwo_{case_name}')

        ######################################################################################
        ##### No need to have this anymore ....
        # # For cases with changing injection horizon
        # if self.inj_hrzn:
        #     self.time_query = list(np.arange(self.inj_hrzn+1)+self.time_start_year)
        #     if self.yr_after_shutin_disp:
        #         for yy in self.yr_after_shutin_disp:
        #             self.time_query.append(self.inj_hrzn+self.time_start_year+yy)
        # else:
        #     print(f"Injection horizon is None, no time query for CMG result extraction ...")
        ######################################################################################

        try:

            x_new, y_new, VERDSPLGEO_arr = cmgrst.rwo_reader2arr(folder=rwo_dir,
                                                                 sim=case_name,
                                                                 prop='Vertical Displacement from Geomechanics',
                                                                 layer_nums=self.layer_nums,
                                                                 time_query=[f'Vertical Displacement from Geomechanics_{t}-Jan-01' for t in self.time_query],
                                                                 x_dir_key=self.x_dir_key, y_dir_key=self.y_dir_key)

            self.cmg2npy = VERDSPLGEO_arr
            self.cmg2npy_x_coord = x_new
            self.cmg2npy_y_coord = y_new
            
            if save == True:
                np.save(os.path.join(self.npy_folder, f"{case_name.split('.')[0]}_VERDSPLGEO.npy"), VERDSPLGEO_arr)
                return True
            else:
                return VERDSPLGEO_arr
            
        except:
            if self.err_stop:
                raise ValueError(f'{case_name} VERDSPLGEO has an error when reading rwo to npy ...')
            else:
                print(f'{case_name} VERDSPLGEO has an error when reading rwo to npy ...')